<a href="https://colab.research.google.com/github/ishgirwan/gee/blob/master/gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
!pip install geopandas
!pip install rasterio
!pip install geemap

In [4]:
import numpy as np
import geopandas as gpd

# visualisation
import matplotlib.pyplot as plt
import geemap
from rasterio.plot import show
import rasterio

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Bengaluru_Rural.cpg to Bengaluru_Rural.cpg
Saving Bengaluru_Rural.dbf to Bengaluru_Rural.dbf
Saving Bengaluru_Rural.prj to Bengaluru_Rural.prj
Saving Bengaluru_Rural.sbn to Bengaluru_Rural.sbn
Saving Bengaluru_Rural.sbx to Bengaluru_Rural.sbx
Saving Bengaluru_Rural.shp to Bengaluru_Rural.shp
Saving Bengaluru_Rural.shp.xml to Bengaluru_Rural.shp.xml
Saving Bengaluru_Rural.shx to Bengaluru_Rural.shx
User uploaded file "Bengaluru_Rural.cpg" with length 5 bytes
User uploaded file "Bengaluru_Rural.dbf" with length 913140 bytes
User uploaded file "Bengaluru_Rural.prj" with length 401 bytes
User uploaded file "Bengaluru_Rural.sbn" with length 12492 bytes
User uploaded file "Bengaluru_Rural.sbx" with length 1124 bytes
User uploaded file "Bengaluru_Rural.shp" with length 2576592 bytes
User uploaded file "Bengaluru_Rural.shp.xml" with length 156955 bytes
User uploaded file "Bengaluru_Rural.shx" with length 9484 bytes


In [7]:
villages = gpd.read_file('Bengaluru_Rural.shp')
villages.sample(5)

,KGISVillag,KGISVill_1,KGISVill_2,KGISHobliI,UniqueVill,Bhucode,CensusVill,BhoomiVill,created_us,created_da,last_edite,last_edi_1,LGDGPCode,LGDGPCode_,SHAPE_Leng,SHAPE_Area,geometry
130,20114,2103020058,Udayagiri,497,2103020058_1,210301002058,2958305603625400,2103020058,SA,2019-04-23,SA,2019-04-23,215424,None,5645.118269,1.333830e+06,"POLYGON ((790450.246 1461805.301, 790378.462 1..."
1005,21011,2101020036,Channohalli,936,2101020036_1,210101002016,2958305601624736,2101020036,SA,2019-04-23,SA,2019-04-23,215591,None,4319.555037,8.446296e+05,"POLYGON ((742346.581 1467412.230, 742418.384 1..."
392,20387,2102040048,Sonnenahalli,231,2102040048_1,210201001060,2958305602625194,2102040048,SA,2019-04-23,SA,2019-04-23,215470,None,4484.187693,4.624012e+05,"POLYGON ((761679.559 1470592.204, 761678.750 1..."
97,20081,2103020034,J.I.Kempalinganahalli,497,2103020034_1,210301002030,2958305603625385,2103020034,SA,2019-04-23,SA,2019-04-23,215436,None,3190.387741,5.693834e+05,"POLYGON ((789024.615 1464991.249, 789020.856 1..."
1099,21106,2101060001,J.I.Lakkappanahalli,985,2101060001_1,210101003029,2958305601624814,2101060001,SA,2019-04-23,SA,2019-04-23,215581,None,5053.439503,1.361698e+06,"POLYGON ((757149.619 1461080.200, 757148.067 1..."


In [8]:
villages.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   KGISVillag  1173 non-null   int64   
 1   KGISVill_1  1173 non-null   object  
 2   KGISVill_2  1173 non-null   object  
 3   KGISHobliI  1173 non-null   int64   
 4   UniqueVill  1173 non-null   object  
 5   Bhucode     1173 non-null   object  
 6   CensusVill  1173 non-null   object  
 7   BhoomiVill  1163 non-null   object  
 8   created_us  1173 non-null   object  
 9   created_da  1173 non-null   object  
 10  last_edite  1173 non-null   object  
 11  last_edi_1  1173 non-null   object  
 12  LGDGPCode   1120 non-null   object  
 13  LGDGPCode_  0 non-null      object  
 14  SHAPE_Leng  1173 non-null   float64 
 15  SHAPE_Area  1173 non-null   float64 
 16  geometry    1173 non-null   geometry
dtypes: float64(2), geometry(1), int64(2), object(12)
memory usage: 155.9+ KB


In [9]:
# Import landsat 8 image collection
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

In [10]:
def get_geometry(roi, df, column_name):
    """
    Returns the feature collection (geometry) for a region of interest
    
    :params roi: Region of Interest
    
    returns - geometry feature collection
    """
    
    # get the polygons for roi
    polygons = (df.geometry[df[column_name] == roi].values)
    # convert each polygon into a feature and append
    features = []
    for p in range(len(polygons)):
        f = ee.Feature(gpd.GeoSeries(polygons[0]).__geo_interface__['features'][0])
        features.append(f)
    
    # convert list of features into feature collection
    fc = ee.FeatureCollection(features)
    return fc

In [11]:
geometry = get_geometry(roi='Injanahalli', df=villages, column_name='KGISVill_2')

In [12]:
geometry.getInfo()

{'columns': {'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[813442.1617450004,
       1446955.306038],
      [813297.2078450004, 1446954.798738],
      [812766.9293450004, 1446983.873738],
      [812599.0951450004, 1446991.870738],
      [812598.5008450004, 1446992.464638],
      [812598.4455450005, 1446991.912338],
      [812597.4568450004, 1446982.069138],
      [812576.3145450004, 1446818.217538],
      [812550.6957450004, 1446677.050038],
      [812544.4653450004, 1446636.941238],
      [812544.4440450005, 1446636.801338],
      [812529.6715450005, 1446543.137138],
      [812519.1478450005, 1446490.009638],
      [812510.7413450005, 1446449.743738],
      [812511.0696450004, 1446448.185638],
      [812512.4721450005, 1446441.525038],
      [812498.3760450005, 1446434.4934379999],
      [812457.2734450004, 1446417.899538],
      [812404.8775450005, 1446399.629738],
      [812369.7951450004, 1446388.861438],
      [812360.8584450004, 1446386.471438],
      [

In [13]:
# interactive mapping using geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [14]:
filtered_collection = l8.filterBounds(geometry).filterDate('2015-01-01','2015-12-31').filterMetadata('CLOUD_COVER','less_than',1)
filtered_collection.size().getInfo()

25334

In [16]:
image = filtered_collection.median()

In [17]:
# clip to geometry
clip = image.clip(geometry).toFloat()

In [18]:
# select params
params = {'bands': ['B4','B3','B2'],
         'gamma': 1,
          'min': 0,
          'max': 0.5
         }
# visualize
Map.centerObject(geometry)
Map.addLayer(clip, params, 'Clip', True, 1)

In [19]:
# creating a task to export clipped image to drive
task = ee.batch.Export.image.toDrive(clip, 
                                    description = 'image',
                                    folder = 'Landsat',                                    
                                    region = geometry.geometry(),
                                    scale = 30,
                                    crs = 'EPSG:3857'                                    
                                    )
task.start()

In [20]:
def check_status(task_id):
    i = 0
    while True:
        if i%30000 == 0:
            status = ee.data.getTaskStatus(task_id)[0]['state']

            print('Status: ', status)
        i+=1
        if status.lower() == 'failed' or status.lower() == 'completed':
            break
check_status(task.id)

Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  RUNNING
Status:  RUNNING
Status:  RUNNING
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  READY
Status:  R

In [ ]:
# viasualizing the image - RGB bands need to be manually named using ColorInterp to view RGB image

plt.figure(figsize=(10,8))
# the file has been downloaded from drive to show here
image = rasterio.open('/content/drive/My Drive/Landsat/test_image.tif')
show(image, adjust='linear')
plt.show()